# SIT319/SIT744 Practical 2: Introduction to TensorBoard and automatic differentiation



ℹ We suggest that you run this notebook using Google Colab.





## Task 1: Use TensorBoard with PyTorch

TensorBoard is a visualization tool that helps monitor PyTorch training, debug models, and compare experiments.


### Step 1: Enable TensorBoard in Colab

Colab already includes TensorBoard, but you can ensure it's installed:

In [32]:
# Load TensorBoard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


If you are running this notebook locally, install TensorBoard in your terminal:

```shell
$ pip install tensorboard
```

### Step 2: Create a TensorBoard logger

TensorBoard looks for a log folder called logdir, which contains summary data to be visualised. In PyTorch, we define logdir when initializing SummaryWriter

In [33]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import time

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir=f"runs/experiment_{int(time.time())}")

The `SummaryWriter` logs data to `runs/experiment_xxx`, which TensorBoard reads.

> 📝 Check what new folders have been created by the above code. Do you see any files inside the folders?

Here is how TensorBoard expects logdir to be organised.


#### runs

A run refers to a separate execution of the model, typically stored as a subfolder under logdir. Multiple runs allow comparison of different experiments.

In [34]:
writer1 = SummaryWriter(log_dir="runs/experiment_1")
writer2 = SummaryWriter(log_dir="runs/experiment_2")

#### event files

PyTorch's SummaryWriter creates event files in logdir.
These files have names like events.out.tfevents.<timestamp>.
These are automatically recognized by TensorBoard.

Each file contains records called *summaries*.




#### tags

You add tags to a summary by passing a `tag` argument in logging calls (See examples below). In TensorBoard, these tags allow you to filter and categorise data to be visualised.

In [35]:
# Logging loss to an event file
for epoch in range(10):
    loss = 0.05 * epoch  # Dummy loss for illustration
    writer.add_scalar("Loss/train", loss, epoch)  # 'Loss/train' is the tag
writer.close()

Let's visualise the dummy loss written into the first SummaryWriter:

In [36]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 84163), started 0:41:12 ago. (Use '!kill 84163' to kill it.)

### Additional exercise:  Logging Dummy Metrics

1. Setup TensorBoard:
 - Import necessary modules and initialize a `SummaryWriter`.
 - Create a log folder (e.g. "runs/experiment_xxx") where event files will be saved.

2. Log a Dummy Loss:

 - Simulate training by iterating over several epochs (e.g. 20).
 - For each epoch, compute a dummy loss (for example, using a simple mathematical function like `loss = 1/(epoch+1)` or any function you choose) and log it using `writer.add_scalar("Loss/train", loss, epoch)`.

3. Visualize in TensorBoard:

 - Run TensorBoard (e.g. `%tensorboard --logdir runs` in Colab or locally) and check that your loss curve is visible.

Additional Challenge:

- Log an additional metric (for instance, “Accuracy/train”) and compare the two curves in TensorBoard.
- Create another run with a different simulated training strategy and compare the two runs side-by-side.


In [37]:
writer = SummaryWriter(log_dir=f"runs/experiment_{int(time.time())}")
# Logging loss to an event file
for epoch in range(20):
    loss = 1/(epoch + 1)  # Dummy loss for illustration
    writer.add_scalar("Loss/train", loss, epoch)  # 'Loss/train' is the tag
writer.close()
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 84163), started 0:41:12 ago. (Use '!kill 84163' to kill it.)

## Task 2: Visualise computational graph of a PyTorch model

Given a PyTorch model, we can use TensorBoard to visualise its Computational Graph.

In [38]:
import torch


# Define a PyTorch model for y = x^2 + x
class FooFunction(nn.Module):
    def forward(self, x):
        return x ** 2 + x  # Function: y = x^2 + x

# Initialize model
model = FooFunction()

# Create input tensor
x = torch.tensor([[2.0]])  # Must be a tensor inside a list for proper graph logging


# Log computational graph to TensorBoard
writer.add_graph(model, input_to_model=[x])
writer.flush()

In [39]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 84163), started 0:41:12 ago. (Use '!kill 84163' to kill it.)

Now you can refresh TensorBoard and you should be able to see a **GRAPHS** tab. There you can find the computation graph. Click inside the `FooFunction` node to understand how the model is implemented.

You should be able to see a **GRAPHS** tab and there you can find the computational graph.
> How many *operations* do you see in the computational graph?

> 📝 **Exercise**:
> 1. Follow the example above to display the computation graph of the function $z = 3 x^2 + 2xy$.
  - Define a simple PyTorch model by subclassing nn.Module.
  - Create a sample input tensor (ensure it’s the right shape, e.g. wrapped in a list if needed).
  - Use `writer.add_graph(model, input_to_model=[x])` to log the model’s computational graph to TensorBoard.
> 2. Log the value of the sine function from 0 to 100; Display the values in TensorBoard. (*Hint: use a different tag.*)
> 3. Create another run. And plot the cosine function instead. (*Hint: use a different summary writer.*)

In [40]:
import torch


# Define a PyTorch model for y = x^2 + x
class FooFunction2(nn.Module):
    def forward(self, x, y):
        return 3* x ** 2 + 2*x*y  # Function: y = x^2 + x

# Initialize model
model = FooFunction2()

# Create input tensor
x = torch.tensor([[2.0]])  # Must be a tensor inside a list for proper graph logging
y = torch.tensor([[3.0]])  # Must be a tensor inside a list for proper graph logging

# Log computational graph to TensorBoard
writer.add_graph(model, input_to_model=[x, y])
writer.flush()

In [41]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 84163), started 0:41:12 ago. (Use '!kill 84163' to kill it.)

## Task 3. PyTorch Automatic Differentiation (Autograd)

Computing gradient is a core requirement for training deep learning models. PyTorch's autograd module enables automatic differentiation, making it easy to compute gradients.

In [ ]:
# Create input tensor with requires_grad=True to track gradients
x = torch.tensor([[2.0]], requires_grad=True)

# Forward pass: Compute y = x^2 + x
y = model(x)

> 📝 What is the value of $y$? Use pen and paper to work out  the gradient $∇y$?

Autograd’s internal gradient functions (`grad_fn`) are added in the forward pass.

In [ ]:
# Print the recorded gradient function in the graph
print(y.grad_fn)  # Shows the last operation in the graph

# Show the chain of operations in the graph
print(y.grad_fn.next_functions)

# Going one level deeper
print(y.grad_fn.next_functions[0][0].next_functions)

> Can you map these `grad_fn` to their respective operation nodes in the computational graph?

In [ ]:
# Compute gradients (dy/dx)
y.backward()

# Print gradient
print(f"Gradient dy/dx at x={x.item()}: {x.grad.item()}")

### Disabling Autograd (No Gradient Tracking)

Sometimes, we don’t need gradients, e.g., during inference.
Disable autograd using `.detach()` or `torch.no_grad()`. Stopping unnecessary gradient tracking saves GPU memory.

In [ ]:
print(y.requires_grad)

# Create a tensor without gradient tracking
z = y.detach()
print(z.requires_grad)

Using `torch.no_grad()`

In [ ]:
with torch.no_grad():
    y = x ** 2
print(y.requires_grad)  # Output: False

**Exercise**: Follow the example above to compute the gradient of function $z(x,y) = 3 x^2 + 2xy$.

Additional exercise: Compute Gradients for a Custom Function

1. Define a Function with Two Inputs:

 - For example, define $z(x,y)=3x^2+2xy$.
 - Create tensors for x and y with requires_grad=True.

2. Compute the Function and Backward Pass:
 - Calculate $z$ using the defined formula.
 - Call `z.backward()` to compute the gradients.
 - Print the gradients of `x` and `y`.

3. Disable Gradient Tracking:

 - Use `torch.no_grad()` or the `.detach()` method to perform an operation without tracking gradients.
 - Verify by checking the `requires_grad` property of the output.

## Additional resources

- [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
- [Visualizing Models, Data, and Training with TensorBoard](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html)
- [A Gentle Introduction to torch.autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)